In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from library import common as cm
from library import plot

In [ ]:
%run setup.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set(style='darkgrid')

Run with PERMUTE=Flase, VIX=False

In [ ]:
dirs_dict = {
    'no hedge': f'{res_dir}Regression/No_Hedge/',
    'BS': f'{res_dir}Regression/BS_Benchmark/',
    'Delta_only': f'{res_dir}Regression/Delta_only/',
    'Vega_only': f'{res_dir}Regression/Vega_only/',
    'Gamma_only': f'{res_dir}Regression/Gamma_only/',
    'Vanna_only': f'{res_dir}Regression/Vanna_only/',
    #'Bias': f'{res_dir}Regression/Bias/',
    'Delta_Gamma': f'{res_dir}Regression/Delta_Gamma/',
    'Delta_Vega': f'{res_dir}Regression/Delta_Vega/',
    'Delta_Vanna': f'{res_dir}Regression/Delta_Vanna/',
    'Delta_Vega_Gamma': f'{res_dir}Regression/Delta_Vega_Gamma/',
    'Delta_Vega_Vanna': f'{res_dir}Regression/Delta_Vega_Vanna/',
    'Delta_Gamma_Vanna': f'{res_dir}Regression/Delta_Gamma_Vanna/',
    'Delta_Vega_Gamma_Vanna': f'{res_dir}Regression/Delta_Vega_Gamma_Vanna/',
    'Hull_White': f'{res_dir}Regression/Hull_White/',
    'Relaxed Hull_White': f'{res_dir}Regression/Hull_White_relaxed/',
    'Network/Normal_Feature': f'{res_dir}Network/Normal_Feature/',
    'Network/Delta_Vega': f'{res_dir}Network/Delta_Vega/',
    'Network/Delta_Vega_Vanna': f'{res_dir}Network/Delta_Vega_Vanna/',
}

In [ ]:
painter = plot.Painter(dirs_dict)
painter.load_aggregate_simulation(NUM_TEST)

In [ ]:
print(f'Load result PNL from {res_dir}')

In [ ]:
%run Load_Clean_aux.py normal
# merging the test sets into one df.
df = pd.DataFrame()
for i, v in enumerate(mc_sets):
    v['testperiod'] = i
    v['index'] = v.index
    df = df.append(v)
df = df.reset_index()
del mc_sets, df_train

#### Plot confidence interval

In [ ]:
painter.plot_coef_err('Delta_only', {'Call': '0', 'Put': '1'}, 'delta_bs')
plt.title('Delta')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Gamma_only', {'Call': '0', 'Put': '1'}, 'gamma_n')
plt.title('Gamma')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Vega_only', {'Call': '0', 'Put': '1'}, 'vega_n')
plt.title('Vega')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Vanna_only', {'Call': '0', 'Put': '1'}, 'vanna_n')
plt.title('Vanna')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Bias', {'Call': '0', 'Put': '1'}, 'bias')
plt.title('Bias')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Delta_Vega_Vanna', {'Call': '0', 'Put': '1'}, 'delta_bs')
plt.title('Delta')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Delta_Vega_Vanna', {'Call': '0', 'Put': '1'}, 'vega_n')
plt.title('Vega')
plt.xlabel('Time window')
plt.ylabel('Coefficient')

In [ ]:
painter.plot_coef_err('Delta_Vega_Vanna', {'Call': '0', 'Put': '1'}, 'vanna_n')
plt.xlabel('Time window')
plt.title('Vanna')
plt.ylabel('Coefficient')

#### Set by set PNL

In [ ]:
kws = [
    ('no hedge', 'Zero hedge', '>', -0.3, sns.color_palette()[4]), 
    ('BS', 'BS-Delta', 'o', -0.2, sns.color_palette()[0]), 
       ('Delta_Vega_Vanna', 'Delta-Vega-Vanna regression', 'x', -0.1, sns.color_palette()[1]), 
       ('Hull_White', 'Hull-White regression', 'v', 0.1, sns.color_palette()[2]), 
       ('Network/Delta_Vega', r'ANN $(\Delta_{\rm BS};\, \mathcal{V}_{\rm BS};\, \tau)$', '^', 0.2, sns.color_palette()[3]),
      ]

In [ ]:
ins = cm.LocalInspector()
ins.plug_existing(painter.pnl)

In [ ]:
err = ins.compare_period()
length = err.shape[0]

In [ ]:
for col in kws:
    plt.plot(np.arange(1, length+1)+col[3], err[col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.yscale('log')
plt.ylim((1e-4, 1e1))   # change range for other datasets
plt.xlabel('Out-of-sample set')
plt.ylabel('Mean squared hedging error')
plt.title('Heston (2 days)')
plt.xticks(np.arange(2, 21, 2))

In [ ]:
err = ins.compare_period('call')
length = err.shape[0]

for col in kws:
    plt.plot(np.arange(1, length+1)+col[3], err[col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.legend(frameon=False)
plt.xlabel('Test set')
plt.ylabel('Mean squared hedging error')

In [ ]:
err = ins.compare_period('put')
length = err.shape[0]

for col in kws:
    plt.plot(np.arange(1, length+1)+col[3], err[col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.legend(frameon=False)
plt.xlabel('Test set')
plt.ylabel('Mean squared hedging error')

#### time-t MSHE
$$MSHE_t = \sum_j\left(\frac{100 V^\delta_{t+1, j}}{S_t}\right)^2 $$
This the the MSHE for each day t.

In [ ]:
daily_mshe = pd.DataFrame()
for key, value in dirs_dict.items():
    temp = painter.pnl[key][['date', 'PNL']].groupby('date').agg([lambda x: np.mean(x**2)])
    daily_mshe[key] = temp[('PNL', '<lambda>')]    

In [ ]:
plt.violinplot(daily_mshe.T, showmeans=False, showmedians=True);

In [ ]:
plt.boxplot(daily_mshe.T);

In [ ]:
zscore_tables = {}
temp = [1.]
for x in temp:
    zscore_tables[x] = pd.DataFrame()

keys = list(dirs_dict.keys())
for i in range(1, len(keys)):
    for j in range(1, i):
        for x in temp:
            zscore_tables[x].loc[keys[i], keys[j]] = cm.get_zscore(daily_mshe, keys[i], keys[j], x)

for key, value in zscore_tables.items():
    value.to_csv(f'{res_dir}truncate_{key}.csv', float_format='%.2f')

In [ ]:
""" Confidnece interval """
keys = list(dirs_dict.keys())

zconf_tables = {}
temp = [1.]
for x in temp:
    rows, cols = keys[1:], keys[1:]
    sub_cols = ['Lower', 'Upper']
    cols_indices = pd.MultiIndex.from_product([cols, sub_cols], names=['Type', 'Bounds'])
    zconf_tables[x] = pd.DataFrame(index=rows, columns=cols_indices)
    
for i in range(1, len(keys)):
    for j in range(1, i):
        for x in temp:
            zconf_tables[x].loc[keys[i], keys[j]] = cm.get_z_confidence(daily_mshe, keys[i], keys[j], x)
            
for key, value in zconf_tables.items():
    file = f'{res_dir}zconf_truncate_{key}_{FREQ}.csv'
    with open(file, 'w+') as f:
        f.write(f'Multiplied by 1*10^5\n')
    (value * 1e5).astype('float').to_csv(f'{res_dir}zconf_truncate_{key}_{FREQ}.csv', float_format='%.2f', mode='a')

### Hedging error vs. sensitivites or features

In [ ]:
if UNDERLYING_MODEL == 'Heston':
    model_name = 'Delta_Vega_Vanna'
elif UNDERLYING_MODEL == 'BS':
    model_name = 'Gamma_only'

In [ ]:
axes = painter.pnl_vs_feature_v2(df, model_name, 'M0', 10, sim_data=True, overall=True, normalized=True, qcut=True, ylog=True,
                         ylims=(5e-4, 3))
axes[0].set_title('Heston (1 day)')
axes[0].set_xlabel(r'Moneyness')
axes[0].set_ylabel(r'Mean squared relative hedging error')
axes[0].set_xticks(axes[0].get_xticks()[:-1])
axes[0].legend(frameon=False)

In [ ]:
axes = painter.pnl_vs_feature_v2(df, model_name, 'tau0', 10, sim_data=True, overall=True, normalized=True, qcut=True, ylog=True,
                         ylims=(5e-4, 3))
axes[0].set_title('Heston (1 day)')
axes[0].set_xlabel(r'Time-to-maturity')
axes[0].set_ylabel(r'Mean squared relative hedging error')
axes[0].set_xticks(axes[0].get_xticks()[:-1])
axes[0].legend(frameon=False)

In [ ]:
axes = painter.pnl_vs_feature_v2(df, model_name, 'delta_bs', 10, sim_data=True, overall=True, normalized=True, qcut=True, 
                          ylog=True, ylims=(5e-4, 3))
axes[0].set_title('Heston (1 day)')
axes[0].set_xlabel(r'Delta')
axes[0].set_ylabel(r'Mean squared relative hedging error')
axes[0].set_xticks(axes[0].get_xticks()[:-1])
axes[0].legend(frameon=False)

In [ ]:
painter.pnl_vs_feature_v2(df, model_name, 'delta_bs', 2, sim_data=True, overall=False, normalized=True, qcut=True, 
                          ylog=True, ylims=(1e-4, 2)
                        )

In [ ]:
axes = painter.pnl_vs_feature_v2(df, model_name, 'gamma_n', 10, sim_data=True, overall=True, 
                  normalized=True, qcut=True, ylog=True, ylims=(5e-4, 3)
                 )
axes[0].set_title('Heston (1 day)')
axes[0].set_xlabel(r'Gamma')
axes[0].set_ylabel(r'Mean squared relative hedging error')
axes[0].set_xticks(axes[0].get_xticks()[:-1])
axes[0].legend(frameon=False)

In [ ]:
painter.pnl_vs_feature_v2(df, model_name, 'gamma_n', 2, sim_data=True, overall=False, 
                  normalized=True, qcut=True, ylog=True, ylims=(5e-4, 3))

In [ ]:
axes = painter.pnl_vs_feature_v2(df, model_name, 'vega_n', 10, sim_data=True, normalized=True, overall=True,
                  qcut=True, ylog=True, ylims=(5e-4, 3))
ticks = axes[0].get_xticks()
tmp = [True] + (ticks[1:] - ticks[:-1] > 0.4).tolist()
a = [round(x, 1) if i is True else None for i, x in zip(tmp, ticks)]
axes[0].set_xticklabels(a[:-1])
axes[0].set_xlabel('Vega')
axes[0].set_ylabel('Mean squared hedging error')
axes[0].set_title('Heston (1 day)')
axes[0].legend(frameon=False)

In [ ]:
axes = painter.pnl_vs_feature_v2(df, model_name, 'vega_n', 2, sim_data=True, normalized=True, overall=False,
                  qcut=True, ylog=True, ylims=(1e-4, 2))